# Setup S3 Bucket for OpenAQ data

In [1]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
s3_private_path_openaq = "s3://{}/aai-final-project/openaq".format(bucket)
print(s3_private_path_openaq)

s3://sagemaker-us-east-1-768099485759/aai-final-project/openaq


8904 is the index of the West Los Angeles sensor

In [3]:
!aws s3 cp --recursive s3://openaq-data-archive/records/csv.gz/locationid=8904/year=2024/ $s3_private_path_openaq

copy: s3://openaq-data-archive/records/csv.gz/locationid=8904/year=2024/month=01/location-8904-20240103.csv.gz to s3://sagemaker-us-east-1-768099485759/aai-final-project/openaq/month=01/location-8904-20240103.csv.gz
copy: s3://openaq-data-archive/records/csv.gz/locationid=8904/year=2024/month=01/location-8904-20240104.csv.gz to s3://sagemaker-us-east-1-768099485759/aai-final-project/openaq/month=01/location-8904-20240104.csv.gz
copy: s3://openaq-data-archive/records/csv.gz/locationid=8904/year=2024/month=01/location-8904-20240102.csv.gz to s3://sagemaker-us-east-1-768099485759/aai-final-project/openaq/month=01/location-8904-20240102.csv.gz
copy: s3://openaq-data-archive/records/csv.gz/locationid=8904/year=2024/month=01/location-8904-20240105.csv.gz to s3://sagemaker-us-east-1-768099485759/aai-final-project/openaq/month=01/location-8904-20240105.csv.gz
copy: s3://openaq-data-archive/records/csv.gz/locationid=8904/year=2024/month=01/location-8904-20240101.csv.gz to s3://sagemaker-us-east

In [4]:
print(s3_private_path_openaq)

s3://sagemaker-us-east-1-768099485759/aai-final-project/openaq


In [5]:
!aws s3 ls $s3_private_path_openaq/

                           PRE month=01/
                           PRE month=02/
                           PRE month=03/
                           PRE month=04/
                           PRE month=05/
                           PRE month=06/
                           PRE month=07/
                           PRE month=08/
                           PRE month=09/
                           PRE month=10/
                           PRE month=11/
                           PRE month=12/


In [6]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/aai-final-project/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

/tmp/ipykernel_125/730034397.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [7]:
from pyathena import connect

In [8]:
# TODO Give this database a better name
database_name = "project_data"
table_name_openaq = "final_project_openaq"

In [9]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [10]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [11]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS aai_aws


In [12]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_125/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [13]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         location_id int,
         sensors_id int,
         location string,
         datetime string,
         latitude float,
         longitude float,
         parameter string,
         unit string,
         value float
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_openaq, s3_private_path_openaq
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS aai_aws.final_project_openaq(
         location_id int,
         sensors_id int,
         location string,
         datetime string,
         latitude float,
         longitude float,
         parameter string,
         unit string,
         value float
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-768099485759/aai-final-project/openaq'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [14]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_125/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [15]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

/tmp/ipykernel_125/4130537117.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [16]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_125/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,final_project_openaq


In [31]:
# TODO Need to remove quotes from string parameters to make queries easier
location_name = '"West Los Angeles - V-8904"'

statement = """SELECT * FROM {}.{} WHERE location = '{}' LIMIT 100""".format(
    database_name, table_name_openaq, location_name
)

print(statement)

SELECT * FROM aai_aws.final_project_openaq WHERE location = '"West Los Angeles - V-8904"' LIMIT 100


In [32]:
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_125/8222894.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,location_id,sensors_id,location,datetime,latitude,longitude,parameter,unit,value
0,8904,25979,"""West Los Angeles - V-8904""","""2024-01-11T01:00:00-08:00""",34.051098,-118.45638,"""o3""","""ppm""",0.0290
1,8904,25979,"""West Los Angeles - V-8904""","""2024-01-11T02:00:00-08:00""",34.051098,-118.45638,"""o3""","""ppm""",0.0290
2,8904,25979,"""West Los Angeles - V-8904""","""2024-01-11T03:00:00-08:00""",34.051098,-118.45638,"""o3""","""ppm""",0.0340
3,8904,25979,"""West Los Angeles - V-8904""","""2024-01-11T04:00:00-08:00""",34.051098,-118.45638,"""o3""","""ppm""",0.0360
4,8904,25979,"""West Los Angeles - V-8904""","""2024-01-11T05:00:00-08:00""",34.051098,-118.45638,"""o3""","""ppm""",0.0330
...,...,...,...,...,...,...,...,...,...
95,8904,4272246,"""West Los Angeles - V-8904""","""2024-01-12T00:00:00-08:00""",34.051098,-118.45638,"""no""","""ppm""",0.0003
96,8904,25978,"""West Los Angeles - V-8904""","""2024-08-30T01:00:00-07:00""",NaN,NaN,"""no2""","""ppm""",NaN
97,8904,25978,"""West Los Angeles - V-8904""","""2024-08-30T02:00:00-07:00""",NaN,NaN,"""no2""","""ppm""",NaN
98,8904,25978,"""West Los Angeles - V-8904""","""2024-08-30T03:00:00-07:00""",NaN,NaN,"""no2""","""ppm""",NaN
